In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [3]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1)
])

In [4]:
def random_batch(X, y, batch_size=32):
    ind = np.random.randint(len(X), size=batch_size)
    return X[ind], y[ind]

In [5]:
n_epochs = 10
batch_size = 32
n_steps = len(X_train) // batch_size
loss_fn = keras.losses.mean_squared_error
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
acc_metric = keras.metrics.MeanAbsoluteError()

In [7]:
for epoch in range(1, n_epochs + 1):
    print(f"\rEpoch {epoch}/{n_epochs}")
    pb_i = keras.utils.Progbar(len(y_train))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as g:
            y_pred = model(X_batch, training=True)
            loss = loss_fn(y_batch, y_pred)
        gradients = g.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        
        acc_metric.update_state(y_batch, y_pred)
        acc = acc_metric.result()
        pb_i.add(batch_size, values=[("acc", acc)])
        
    print(f"\n{acc_metric.name} - {acc_metric.result()}")
    acc_metric.reset_states()
    

Epoch 1/10
Epoch 2/100 [============================>.] - ETA: 0s - acc: 0.9092
Epoch 3/100 [============================>.] - ETA: 0s - acc: 0.9191
Epoch 4/100 [============================>.] - ETA: 0s - acc: 0.9171
Epoch 5/100 [============================>.] - ETA: 0s - acc: 0.9233
Epoch 6/100 [============================>.] - ETA: 0s - acc: 0.8995
Epoch 7/100 [============================>.] - ETA: 0s - acc: 0.9087
Epoch 8/100 [============================>.] - ETA: 0s - acc: 0.9284
Epoch 9/100 [============================>.] - ETA: 0s - acc: 0.9185
Epoch 10/10 [============================>.] - ETA: 0s - acc: 0.8976
11488/11610 [============================>.] - ETA: 0s - acc: 0.8982